# <font color='red'>PyMongo</font>: *(Nested Data)*: Query | Aggregate | Map-Reduce

**Mr Fugu Data Science**

# (◕‿◕✿)

[youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber) | [Gihub](https://github.com/MrFuguDataScience)


# Outcome & Expectations:

+ Create `Aggregate` function
    + Create Map-Reduce functions
        + Describe use cases for both
        

In [ ]:
''' SIDE NOTE: IF YOU ARE SETTING UP A CONNECTION EVERYTIME, LIKE I AM. 
Then you will need to turn on (mongod) so you can send data and do queries'''

In [6]:
import pymongo as pym      # Interface with Python <--> MongoDB 
import pandas as pd        # Create Dataframe
import pprint as pp        # readibility for our data
from bson.code import Code # Help with Map-Reduce

In [12]:
# Making a Connection to MongoClient
client = pym.MongoClient('mongodb://localhost:27017/')

# CREATING A DATABASE:
db = client["berkeley"]

# CREATING A COLLECTION (*AKA* TABLE):
candidate_user_info= db["recruiter_clients"]

# Our Data: first 2 entries

`{[{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Machine Learning',
   'experience': 'Mid',
   'relocation': 'no'}},
 {'candidate': {'first_name': 'Michael',
   'last_name': 'Carter',
   'skills': ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
   'state': 'AR',
   'specialty': 'Machine Learning',
   'experience': 'Junior',
   'relocation': 'yes'}}]`

In [13]:
# Basics Query of Nested Doc:

for i in candidate_user_info.find({'candidate.first_name':'Brenda'}):
    pp.pprint(i)

{'_id': ObjectId('5eb5cc5b47c72ccce7539c1d'),
 'candidate': {'experience': 'Mid',
               'first_name': 'Brenda',
               'last_name': 'Tyler',
               'relocation': 'no',
               'skills': ['Spark'],
               'specialty': 'Database',
               'state': 'UT'}}
{'_id': ObjectId('5eb5cc5b47c72ccce7539d13'),
 'candidate': {'experience': 'Senior',
               'first_name': 'Brenda',
               'last_name': 'Kennedy',
               'relocation': 'no',
               'skills': ['TensorFlow'],
               'specialty': 'Data Visualization',
               'state': 'NM'}}


In [16]:
# Find only people with the skills of Spark:

only_spark=[]
for i in candidate_user_info.find({'candidate.skills':['Spark']},{'_id':0}):
#     pp.pprint(i)
    only_spark.append(i)
only_spark[:3]

[{'candidate': {'first_name': 'Brenda',
   'last_name': 'Tyler',
   'skills': ['Spark'],
   'state': 'UT',
   'specialty': 'Database',
   'experience': 'Mid',
   'relocation': 'no'}},
 {'candidate': {'first_name': 'Tiffany',
   'last_name': 'Olson',
   'skills': ['Spark'],
   'state': 'MO',
   'specialty': 'Statistics',
   'experience': 'Mid',
   'relocation': 'maybe'}},
 {'candidate': {'first_name': 'Travis',
   'last_name': 'Hawkins',
   'skills': ['Spark'],
   'state': 'CO',
   'specialty': 'Statistics',
   'experience': 'Senior',
   'relocation': 'yes'}}]

# Create Dataframe from nested data, and query with `$or` :

In [17]:
# Find people who live in either: California or Florida

store_CA_FL=[]
for i in candidate_user_info.find({'$or' :[{'candidate.state':'CA'},
                                           {'candidate.state':'FL'}]}):
    store_CA_FL.append(i)

# pd.DataFrame.from_dict(g['candidate'],orient='columns')
nested_candidates=[]
only_ids=[]
for i in store_CA_FL:
    nested_candidates.append(i['candidate'])
    only_ids.append(i['_id'])


recruiter_candidates=pd.DataFrame(nested_candidates) 

recruiter_candidates['_id']=only_ids
recruiter_candidates.head()
# store_CA_FL

,first_name,last_name,skills,state,specialty,experience,relocation,_id
0,Joseph,King,"[skLearn, SQL, R, Spark, Java, C++, Python, Te...",FL,Statistics,Senior,maybe,5eb5cc5b47c72ccce7539c1e
1,Tammy,Miller,"[skLearn, R, TensorFlow, MongoDB, C++, Java]",FL,Data Visualization,Mid,maybe,5eb5cc5b47c72ccce7539c5d
2,Michael,Carter,"[Java, C++]",FL,Database,Senior,maybe,5eb5cc5b47c72ccce7539c67
3,Sharon,Taylor,"[MongoDB, R]",CA,Data Visualization,Senior,no,5eb5cc5b47c72ccce7539c73
4,Gloria,Rivera,"[R, SQL, Java, MongoDB, skLearn, Python, Tenso...",CA,Statistics,Senior,maybe,5eb5cc5b47c72ccce7539c82


In [18]:
num_skills_by_person= candidate_user_info.aggregate([{'$project':{'candidate.first_name':1,'candidate.last_name':1,
                                                    'numberOfSkills':{'$cond':{'if':{'$isArray':'$candidate.skills'},
                                                    'then':{'$size':'$candidate.skills'},'else':'NA'}},'_id':0}}])
num_skills_each=[]
for i in num_skills_by_person:
    num_skills_each.append(i)
num_skills_each[:6]

[{'candidate': {'first_name': 'Margaret', 'last_name': 'Mcdonald'},
  'numberOfSkills': 6},
 {'candidate': {'first_name': 'Michael', 'last_name': 'Carter'},
  'numberOfSkills': 6},
 {'candidate': {'first_name': 'Brenda', 'last_name': 'Tyler'},
  'numberOfSkills': 1},
 {'candidate': {'first_name': 'Joseph', 'last_name': 'King'},
  'numberOfSkills': 8},
 {'candidate': {'first_name': 'Laura', 'last_name': 'Webb'},
  'numberOfSkills': 6},
 {'candidate': {'first_name': 'Cheryl', 'last_name': 'Ramirez'},
  'numberOfSkills': 9}]

In [25]:
import pprint as pp
b=[]
for i in candidate_user_info.aggregate( [{'$match': {'candidate.specialty':'Machine Learning'}}]):
#     pp.pprint(i)
    b.append(i)
b[:2]

[{'_id': ObjectId('5eb5cc5b47c72ccce7539c1f'),
  'candidate': {'first_name': 'Laura',
   'last_name': 'Webb',
   'skills': ['TensorFlow', 'C++', 'SQL', 'Java', 'R', 'MongoDB'],
   'state': 'WY',
   'specialty': 'Machine Learning',
   'experience': 'Junior',
   'relocation': 'maybe'}},
 {'_id': ObjectId('5eb5cc5b47c72ccce7539c21'),
  'candidate': {'first_name': 'Charles',
   'last_name': 'Stewart',
   'skills': ['MongoDB', 'C++', 'Java', 'SQL', 'R', 'Python', 'skLearn'],
   'state': 'NM',
   'specialty': 'Machine Learning',
   'experience': 'Mid',
   'relocation': 'maybe'}}]

In [26]:


from bson.code import Code

mapFunc = Code("function(){emit(this.candidate.state,{count:1});}")

redFunc = Code("function(state,val){var value = {count:0};for(i=0; i<val.length;i++){value.count += val[i].count;}return value;}")

map_red=candidate_user_info.map_reduce(mapFunc,redFunc,'skills_byState') 
#skills_byState is stored as a collection in MongoDB

state_counts=[]
for i in map_red.find():
    state_counts.append(i)
state_counts[:6]          

[{'_id': 'AK', 'value': {'count': 10.0}},
 {'_id': 'AL', 'value': {'count': 12.0}},
 {'_id': 'AR', 'value': {'count': 8.0}},
 {'_id': 'AZ', 'value': {'count': 8.0}},
 {'_id': 'CA', 'value': {'count': 11.0}},
 {'_id': 'CO', 'value': {'count': 10.0}}]

In [27]:
# Showing the alternate way of obtaining the data from Map-Reduce:(BY value count)
alt_state=[]
for i in db.skills_byState.find().sort('value.count'):
    alt_state.append(i)
alt_state[:8]

[{'_id': 'VT', 'value': {'count': 2.0}},
 {'_id': 'SD', 'value': {'count': 3.0}},
 {'_id': 'VA', 'value': {'count': 5.0}},
 {'_id': 'NH', 'value': {'count': 6.0}},
 {'_id': 'IN', 'value': {'count': 7.0}},
 {'_id': 'KY', 'value': {'count': 7.0}},
 {'_id': 'NJ', 'value': {'count': 7.0}},
 {'_id': 'TN', 'value': {'count': 7.0}}]

# Citations & Help:

https://api.mongodb.com/python/2.0/examples/map_reduce.html

https://pymongo.readthedocs.io/en/stable/examples/aggregation.html
